#Excercise of Bayes


**Data gồm khoảng 2200 bài báo và có 5 category : business, entertainment, politics, sport và tech. Sau khi fit bằng bayes, check bằng data test thì độ chính xác khoảng 96% **

Link Data thô : https://drive.google.com/drive/folders/1otI7Xhbk-Jzjg7fiNVSukbL1_c8R8FTw?usp=sharing

Link Data đã xử lí thành table: https://drive.google.com/drive/folders/1EInZH9jzRhuPOfIRB44UANZWomk-P6uM?usp=sharing

In [0]:
import numpy as np
import re
import pickle
news_of_train = [469,355,375,460,360]
news_of_test = [41,31,42,51,41]

##Processes data to table and save

In [0]:
def pre_data_1(num_data, dir_name):
	C = len(num_data)
	data = []
	label = []
	for i in range(C):
		for j in range(num_data[i]):
			link = dir_name + '\\' + str(i) + '\\' + str(j) +'.txt'
			f = open(link, 'r', encoding='utf-8', errors='ignore')
			s = f.read().lower()
			s = re.sub(r'[\d\W\t\n\r\f\v]', ' ', s )
			s = re.sub(' +', ' ', s)
			data.append(s.split(' '))
			label.append(i)
	return data, label

def save_list(file_name, list):
	with open(file_name,'wb') as f :
		pickle.dump(list,f)

def get_table(data, label, column):
	x = []
	y = []
	for i in range(len(data)):
		t = []
		for j in range(len(column)):
			if column[j] in data[i] :
				t.append(1)
			else : t.append(0)
		x.append(t)
		y.append(label[i])

	return np.array(x), np.array(y)

def pre_data():
	global news_of_train,news_of_test,dir
	data_train,label_train = pre_data_1(news_of_train,'bbc\\train')
	data_test,label_test = pre_data_1(news_of_test,'bbc\\test')

	column = list(set([item for sublist in data_train for item in sublist]))# list of word
	column = column[1:]# remove word ""

	x_train, y_train = get_table(data_train,label_train,column)
	x_test,y_test = get_table(data_test,label_test,column)	

	save_list('list\\x_train.pkl',x_train)
	save_list('list\\y_train.pkl',y_train)
	save_list('list\\x_test.pkl',x_test)
	save_list('list\\y_test.pkl',y_test)
	save_list('list\\column.pkl',column)

## Class Bayes

In [0]:
class Bayes:
	def fit(self,x, y):
		self.C = len(set(y))
		self.D = len(x[0])
		self.theta = np.zeros((self.C,self.D))
		self.prior = np.zeros(self.C)
		for c in range(self.C) :
			x_in_c = x[y == c, :]
			Non = np.sum(x_in_c, axis = 0 )
			n_c = len(x_in_c)
			self.prior[c] = n_c / len(x)
			self.theta[c,:] = Non / n_c

	def predict(self, x,y):
		epxi = 0.00000000001
		count_correct = 0
		num_test = len(x)
		for i in range(num_test):
			loglik = np.zeros(self.C) 
			for c in range(self.C):
				theta = self.theta[c]
				bitmask = x[i]
				loglik[c] = np.log(self.prior[c] + epxi) +  np.sum( np.dot(bitmask, np.log(theta + epxi)) + np.dot(1-bitmask, np.log(1-theta + epxi)) )
			
			y_hat = np.argmax(loglik)
			if (y_hat == y[i]) :
				count_correct += 1

		return count_correct / num_test


## Main

In [0]:
dic = load_data('list\\column.pkl')
x_train = load_data('list\\x_train.pkl')
y_train = load_data('list\\y_train.pkl')
x_test = load_data('list\\x_test.pkl')
y_test = load_data('list\\y_test.pkl')
b =  Bayes()
b.fit(x_train,y_train)
print('length of dictionary : %d words' %len(dic))
print('train data : %d rows' %len(y_train))
print('test data : %d rows' %len(y_test))
print('test accurary : %f'  % b.predict(x_test,y_test))
